In [8]:
pip install delta-spark

Note: you may need to restart the kernel to use updated packages.


In [9]:
#Install Dependencies
import sys
!{sys.executable} -m pip install hdfs

In [1]:
from os import PathLike
from hdfs import InsecureClient
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import LongType, StringType, IntegerType, StructField, StructType, DoubleType
from delta import *

In [2]:
# warehouse_location points to the default location for managed databases and tables
warehouse_location = 'hdfs://hdfs-nn:9000/warehouse'

#para usarmos o delta tempos de por as ultimas linhas do config para importar bibliotecas
builder = SparkSession \
    .builder \
    .appName("Python Spark SQL Hive integration example") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:1.0.0") \
    .enableHiveSupport() \

spark = spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [28]:
# you can choose any location in HDFS, just be organized 
# Your data lake will grow with time and will become a swamp
spark.sql(
    """
    CREATE DATABASE PL7G1_SILVER LOCATION 'hdfs://hdfs-nn:9000/Poluicao_ar/Silver/PL7G1_SILVER.db'
    """
)

DataFrame[]

In [27]:
spark.sql(
    """
    DROP DATABASE IF EXISTS PL7G1_SILVER CASCADE 
    """
)

DataFrame[]

In [6]:
spark.sql(
    """
    SHOW DATABASES
    """
).show()

+---------------+
|      namespace|
+---------------+
|      accidents|
|         bronze|
|        default|
|           gold|
|   pl7g1_silver|
|severity_silver|
|         silver|
+---------------+



In [29]:
#estrutura final de como a tabela tem de ficar
spark.sql(
    """
    CREATE EXTERNAL TABLE PL7G1_SILVER.air_quality (
        Unique_ID INT,
        Indicator_ID INT,
        Name VARCHAR(100),
        Measure VARCHAR(75),
        Measure_Info VARCHAR(50),
        Geo_Type_Name VARCHAR(50),
        Geo_Join_ID INT,
        Geo_Place_Name VARCHAR(100),
        Data_Value DOUBLE,
        Borough VARCHAR (50),
        Month VARCHAR (10)
        
    )
    USING DELTA
    PARTITIONED BY (
        Year VARCHAR(5)
    )
    LOCATION 'hdfs://hdfs-nn:9000/Poluicao_ar/Silver/PL7G1_SILVER.db/air_quality/'
    """
)

DataFrame[]

In [26]:
spark.sql(
    """
    DROP TABLE IF EXISTS PL7G1_SILVER.air_quality
    """
)

DataFrame[]

In [9]:
spark.sql(
    """
    SHOW TABLES FROM PL7G1_SILVER
    """
).show()

+------------+-----------+-----------+
|    database|  tableName|isTemporary|
+------------+-----------+-----------+
|pl7g1_silver|air_quality|      false|
+------------+-----------+-----------+



In [10]:
spark.sql(
    """
    SELECT *
    FROM PL7G1_SILVER.air_quality
    """
).show()

+---------+------------+----+-------+------------+-------------+-----------+--------------+----------+-------+-----+----+
|Unique_ID|Indicator_ID|Name|Measure|Measure_Info|Geo_Type_Name|Geo_Join_ID|Geo_Place_Name|Data_Value|Borough|Month|Year|
+---------+------------+----+-------+------------+-------------+-----------+--------------+----------+-------+-----+----+
+---------+------------+----+-------+------------+-------------+-----------+--------------+----------+-------+-----+----+



In [30]:

#.option("inferSchema", True) \
 

hdfs_path = "hdfs://hdfs-nn:9000//Poluicao_ar/Bronze/Air_Quality.csv"
customSchema = StructType([
    StructField("Unique_ID", IntegerType(), True),
    StructField("Indicator_ID", IntegerType(), True),
    StructField("Name", StringType(), True),
    StructField("Measure", StringType(), True),
    StructField("Measure_Info", StringType(), True),
    StructField("Geo_Type_Name", StringType(), True),
    StructField("Geo_Join_ID", IntegerType(), True),
    StructField("Geo_Place_Name", StringType(), True),
    StructField("Time_Period", StringType(), True),
    StructField("Start_Date", StringType(), True),
    StructField("Data_Value", DoubleType(), True),
    StructField("Borough", StringType(), True)
    
    
])

air = spark \
            .read \
            .option("header",True) \
            .option("delimiter",",") \
            .schema(customSchema) \
            .csv(hdfs_path)

#Criar o Year: Start_Date em Start_Date e Year
from pyspark.sql.functions import when, col, concat, lit, split
air= air.withColumn("Year",split(col("Start_Date"),"/").getItem(2)) 
air= air.withColumn("Month",split(col("Start_Date"),"/").getItem(0)) 




air.toPandas()


Unique_ID  Indicator_ID                  Name Measure Measure_Info  \
0         216498           386            Ozone (O3)    Mean          ppb   
1         216499           386            Ozone (O3)    Mean          ppb   
2         219969           386            Ozone (O3)    Mean          ppb   
3         219970           386            Ozone (O3)    Mean          ppb   
4         164876           383  Sulfur Dioxide (SO2)    Mean          ppb   
...          ...           ...                   ...     ...          ...   
16117     671118           386            Ozone (O3)    Mean          ppb   
16118     671119           386            Ozone (O3)    Mean          ppb   
16119     671120           386            Ozone (O3)    Mean          ppb   
16120     671121           386            Ozone (O3)    Mean          ppb   
16121     671122           386            Ozone (O3)    Mean          ppb   

      Geo_Type_Name  Geo_Join_ID                          Geo_Place_Name  \
0                CD          313                     Coney Island (CD13)   
1                CD          313                     Coney Island (CD13)   
2           Borough            1                                   Bronx   
3           Borough            1                                   Bronx   
4                CD          211        Morris Park and Bronxdale (CD11)   
...             ...          ...                                     ...   
16117            CD          306    Park Slope and Carroll Gardens (CD6)   
16118            CD          305   East New York and Starrett City (CD5)   
16119            CD          304                          Bushwick (CD4)   
16120            CD          303                Bedford Stuyvesant (CD3)   
16121            CD          302  Fort Greene and Brooklyn Heights (CD2)   

          Time_Period  Start_Date  Data_Value Borough  Year Month  
0         Summer 2013  06/01/2013       34.64    None  2013    06  
1         Summer 2014  06/01/2014       33.22    None  2014    06  
2         Summer 2013  06/01/2013       31.25    None  2013    06  
3         Summer 2014  06/01/2014       31.15    None  2014    06  
4      Winter 2008-09  12/01/2008        5.89    None  2008    12  
...               ...         ...         ...     ...   ...   ...  
16117     Summer 2020  06/01/2020       28.70    None  2020    06  
16118     Summer 2020  06/01/2020       29.56    None  2020    06  
16119     Summer 2020  06/01/2020       29.65    None  2020    06  
16120     Summer 2020  06/01/2020       29.28    None  2020    06  
16121     Summer 2020  06/01/2020       28.93    None  2020    06  

[16122 rows x 14 columns]

In [31]:
#meter o nome do mês, 
from pyspark.sql.functions import when, col, concat, lit, split

air= air.withColumn("Month", \
   when((air.Month == "01"), lit("January")) \
     .when((air.Month == "03"), lit("March")) \
     .when((air.Month == "04"), lit("April")) \
     .when((air.Month == "05"), lit("May")) \
     .when((air.Month == "06"), lit("June")) \
     .when((air.Month == "07"), lit("July")) \
     .when((air.Month == "08"), lit("August")) \
     .when((air.Month == "09"), lit("September")) \
     .when((air.Month == "12"), lit("December"))
) 
air.toPandas()

Unique_ID  Indicator_ID                  Name Measure Measure_Info  \
0         216498           386            Ozone (O3)    Mean          ppb   
1         216499           386            Ozone (O3)    Mean          ppb   
2         219969           386            Ozone (O3)    Mean          ppb   
3         219970           386            Ozone (O3)    Mean          ppb   
4         164876           383  Sulfur Dioxide (SO2)    Mean          ppb   
...          ...           ...                   ...     ...          ...   
16117     671118           386            Ozone (O3)    Mean          ppb   
16118     671119           386            Ozone (O3)    Mean          ppb   
16119     671120           386            Ozone (O3)    Mean          ppb   
16120     671121           386            Ozone (O3)    Mean          ppb   
16121     671122           386            Ozone (O3)    Mean          ppb   

      Geo_Type_Name  Geo_Join_ID                          Geo_Place_Name  \
0                CD          313                     Coney Island (CD13)   
1                CD          313                     Coney Island (CD13)   
2           Borough            1                                   Bronx   
3           Borough            1                                   Bronx   
4                CD          211        Morris Park and Bronxdale (CD11)   
...             ...          ...                                     ...   
16117            CD          306    Park Slope and Carroll Gardens (CD6)   
16118            CD          305   East New York and Starrett City (CD5)   
16119            CD          304                          Bushwick (CD4)   
16120            CD          303                Bedford Stuyvesant (CD3)   
16121            CD          302  Fort Greene and Brooklyn Heights (CD2)   

          Time_Period  Start_Date  Data_Value Borough  Year     Month  
0         Summer 2013  06/01/2013       34.64    None  2013      June  
1         Summer 2014  06/01/2014       33.22    None  2014      June  
2         Summer 2013  06/01/2013       31.25    None  2013      June  
3         Summer 2014  06/01/2014       31.15    None  2014      June  
4      Winter 2008-09  12/01/2008        5.89    None  2008  December  
...               ...         ...         ...     ...   ...       ...  
16117     Summer 2020  06/01/2020       28.70    None  2020      June  
16118     Summer 2020  06/01/2020       29.56    None  2020      June  
16119     Summer 2020  06/01/2020       29.65    None  2020      June  
16120     Summer 2020  06/01/2020       29.28    None  2020      June  
16121     Summer 2020  06/01/2020       28.93    None  2020      June  

[16122 rows x 14 columns]

In [32]:
from pyspark.sql.functions import when, col, concat, lit, split
air= air.withColumn("Borough", \
   when((air.Geo_Type_Name == "CD") & (air.Geo_Join_ID >= 101) & (air.Geo_Join_ID <= 112), lit("Manhattan") ) \
     .when((air.Geo_Type_Name == "CD") & (air.Geo_Join_ID >= 201) & (air.Geo_Join_ID <= 212), lit("Bronx")) \
     .when((air.Geo_Type_Name == "CD") & (air.Geo_Join_ID >= 301) & (air.Geo_Join_ID <= 318), lit("Brooklyn")) \
     .when((air.Geo_Type_Name == "CD") & (air.Geo_Join_ID >= 401) & (air.Geo_Join_ID <= 414), lit("Queens")) \
     .when((air.Geo_Type_Name == "CD") & (air.Geo_Join_ID >= 501) & (air.Geo_Join_ID <= 503), lit("Staten Island")) \
                    
     .when((air.Geo_Type_Name == "UHF42") & (air.Geo_Join_ID >= 101) & (air.Geo_Join_ID <= 107), lit("Bronx")) \
     .when((air.Geo_Type_Name == "UHF42") & (air.Geo_Join_ID >= 201) & (air.Geo_Join_ID <= 211), lit("Brooklyn")) \
     .when((air.Geo_Type_Name == "UHF42") & (air.Geo_Join_ID >= 301) & (air.Geo_Join_ID <= 310), lit("Manhattan")) \
     .when((air.Geo_Type_Name == "UHF42") & (air.Geo_Join_ID >= 401) & (air.Geo_Join_ID <= 410), lit("Queens")) \
     .when((air.Geo_Type_Name == "UHF42") & (air.Geo_Join_ID >= 501) & (air.Geo_Join_ID <= 504), lit("Staten Island")) \
                    
     .when((air.Geo_Type_Name == "UHF34") & (air.Geo_Join_ID >= 101) & (air.Geo_Join_ID <= 107), lit("Bronx")) \
     .when((air.Geo_Type_Name == "UHF34") & (air.Geo_Join_ID >= 201) & (air.Geo_Join_ID <= 211), lit("Brooklyn")) \
     .when((air.Geo_Type_Name == "UHF34") & (air.Geo_Join_ID >= 301) & (air.Geo_Join_ID <= 310), lit("Manhattan")) \
     .when((air.Geo_Type_Name == "UHF34") & (air.Geo_Join_ID >= 401) & (air.Geo_Join_ID <= 410), lit("Queens")) \
     .when((air.Geo_Type_Name == "UHF34") & (air.Geo_Join_ID >= 501) & (air.Geo_Join_ID <= 504), lit("Staten Island")) \
                    
     .when((air.Geo_Type_Name == "Borough") & (air.Geo_Join_ID == 1), lit("Bronx")) \
     .when((air.Geo_Type_Name == "Borough") & (air.Geo_Join_ID == 2), lit("Brooklyn")) \
     .when((air.Geo_Type_Name == "Borough") & (air.Geo_Join_ID == 3), lit("Manhattan")) \
     .when((air.Geo_Type_Name == "Borough") & (air.Geo_Join_ID == 4), lit("Queens")) \
     .when((air.Geo_Type_Name == "Borough") & (air.Geo_Join_ID == 5), lit("Staten Island")) \
                    
     .when((air.Geo_Type_Name == "Citywide") & (air.Geo_Join_ID == 1), lit("Citywide")) \
     .when((air.Geo_Type_Name == "Citywide"), lit("Citywide")) \

     .otherwise(lit("Sem Informação")) \
  )

In [33]:
air.show()

+---------+------------+--------------------+-------+------------+-------------+-----------+--------------------+--------------+----------+----------+--------+----+--------+
|Unique_ID|Indicator_ID|                Name|Measure|Measure_Info|Geo_Type_Name|Geo_Join_ID|      Geo_Place_Name|   Time_Period|Start_Date|Data_Value| Borough|Year|   Month|
+---------+------------+--------------------+-------+------------+-------------+-----------+--------------------+--------------+----------+----------+--------+----+--------+
|   216498|         386|          Ozone (O3)|   Mean|         ppb|           CD|        313| Coney Island (CD13)|   Summer 2013|06/01/2013|     34.64|Brooklyn|2013|    June|
|   216499|         386|          Ozone (O3)|   Mean|         ppb|           CD|        313| Coney Island (CD13)|   Summer 2014|06/01/2014|     33.22|Brooklyn|2014|    June|
|   219969|         386|          Ozone (O3)|   Mean|         ppb|      Borough|          1|               Bronx|   Summer 2013|06

In [34]:
from pyspark.sql.functions import *

air=air.select("Unique_ID","Indicator_ID","Name","Measure","Measure_Info","Geo_Type_Name","Geo_Join_ID","Geo_Place_Name","Year","Data_Value", "Borough", "Month") 


In [35]:
air.printSchema()

root
 |-- Unique_ID: integer (nullable = true)
 |-- Indicator_ID: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Measure: string (nullable = true)
 |-- Measure_Info: string (nullable = true)
 |-- Geo_Type_Name: string (nullable = true)
 |-- Geo_Join_ID: integer (nullable = true)
 |-- Geo_Place_Name: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Data_Value: double (nullable = true)
 |-- Borough: string (nullable = false)
 |-- Month: string (nullable = true)



In [36]:
#save to hive delta table
air \
    .select ("Unique_ID","Indicator_ID","Name", "Measure", "Measure_Info", "Geo_Type_Name","Geo_Join_ID","Geo_Place_Name","Year","Data_Value", "Borough", "Month" ) \
    .write \
    .mode("overwrite") \
    .partitionBy("Year") \
    .format("delta") \
    .save("hdfs://hdfs-nn:9000/Poluicao_ar/Silver/PL7G1_SILVER.db/air_quality/")

In [38]:
spark.sql(
    """
    SELECT *
    FROM PL7G1_SILVER.air_quality
    """
).toPandas()

Unique_ID  Indicator_ID                                   Name  \
0         179718           642  Boiler Emissions- Total NOx Emissions   
1         179719           642  Boiler Emissions- Total NOx Emissions   
2         179720           642  Boiler Emissions- Total NOx Emissions   
3         179721           642  Boiler Emissions- Total NOx Emissions   
4         179722           642  Boiler Emissions- Total NOx Emissions   
...          ...           ...                                    ...   
16117     671118           386                             Ozone (O3)   
16118     671119           386                             Ozone (O3)   
16119     671120           386                             Ozone (O3)   
16120     671121           386                             Ozone (O3)   
16121     671122           386                             Ozone (O3)   

              Measure Measure_Info Geo_Type_Name  Geo_Join_ID  \
0      Number per km2       number         UHF42          504   
1      Number per km2       number         UHF42          503   
2      Number per km2       number         UHF42          501   
3      Number per km2       number         UHF42          502   
4      Number per km2       number         UHF42          410   
...               ...          ...           ...          ...   
16117            Mean          ppb            CD          306   
16118            Mean          ppb            CD          305   
16119            Mean          ppb            CD          304   
16120            Mean          ppb            CD          303   
16121            Mean          ppb            CD          302   

                               Geo_Place_Name  Data_Value        Borough  \
0                   South Beach - Tottenville        2.00  Staten Island   
1                                 Willowbrook        2.10  Staten Island   
2                               Port Richmond        2.80  Staten Island   
3                      Stapleton - St. George        4.60  Staten Island   
4                                   Rockaways        6.10         Queens   
...                                       ...         ...            ...   
16117    Park Slope and Carroll Gardens (CD6)       28.70       Brooklyn   
16118   East New York and Starrett City (CD5)       29.56       Brooklyn   
16119                          Bushwick (CD4)       29.65       Brooklyn   
16120                Bedford Stuyvesant (CD3)       29.28       Brooklyn   
16121  Fort Greene and Brooklyn Heights (CD2)       28.93       Brooklyn   

         Month  Year  
0      January  2015  
1      January  2015  
2      January  2015  
3      January  2015  
4      January  2015  
...        ...   ...  
16117     June  2020  
16118     June  2020  
16119     June  2020  
16120     June  2020  
16121     June  2020  

[16122 rows x 12 columns]